## Tutorial: Generating a road network graph and plotting a map using Openstreetmap data
### Part 2: From Pandas Dataframe to graph

#### Environment:
- Python: 2.7
    - Create a Conda virtual environment with python 2.7 [tutorial](https://uoa-eresearch.github.io/eresearch-cookbook/recipe/2014/11/20/conda/)
    - Inside the virtualenv I did a fresh install of Jupyter Notebook, installed a new jupyter kernel and launched this notebook in this new kernel

#### Major Dependencies: 

- Basemap 

    - Basemap, can only be installed using [Conda](https://docs.conda.io/en/latest/miniconda.html)
    - Install Basemap inside the conda virtual env
    
    - (Installing Basemap was a big headache for me)
    - One alternative is to build from source using the command: pip install https://github.com/matplotlib/basemap/archive/master.zip
    
#### Estimated Completion time:
- 2 hours

In [1]:
!python -V

Python 2.7.16 :: Anaconda, Inc.


In [7]:
# Resolving these issues is going to be dependent on your specific system

import os
os.environ["PROJ_LIB"] = os.path.join(os.environ["CONDA_PREFIX"], "share", "proj")

In [8]:
import mpl_toolkits
mpl_toolkits.__path__.append('/Users/bibekpoudel/miniconda/miniconda3/envs/myenv2/lib/python2.7/site-packages/mpl_toolkits/')
from mpl_toolkits.basemap import Basemap

ModuleNotFoundError: No module named 'pyproj'

In [9]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

/Users/bibekpoudel/.pyenv/versions/3.9.1/lib/python3.9/site-packages/pandas/compat/__init__.py:97: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


## Step 1: Global Projection
- Mapping is not perfect, why? [Wiki](https://en.wikipedia.org/wiki/Map_projection)
    - We have to project earths surface to a rectangular area
    - The earth is not a perfect sphere
- So there are many projection models (Different models have different assumptions and trade offs) 
- The projection I have chosen here is Cylindrical Equal Area (CEA) 

- What does projection do?
    - Convert a point in earths surface (lat long) coordinates to rectangular coordinates (x,y)

- Basemap, a matplotlib tool provides many projections [Site](https://matplotlib.org/basemap/users/index.html)

In [10]:
mp = Basemap(llcrnrlon = -118.6983,
            llcrnrlat = 33.8017,
            urcrnrlon = -117.7269,
            urcrnrlat = 34.5720,
            projection='cea',
            resolution='h')

# considering x and y of the entire globe
global_x_and_y=[]

for i, (way_id, n_ids, new_n_ids, lat_long) in new_nodes_ids_df.iterrows():
    insert=[]
    for j in lat_long:
        
        lat = j[0]
        
        long = j[1]
        
        # Long/ Lat is supplied and Long/Lat is returned
        y,x = mp(long, lat)
        
        y = round(y,5)
        x = round(x,5)
        
        # inserted as x and y
        insert.append((x,y))
        
    global_x_and_y.append(insert)
new_nodes_ids_df.insert(loc=4, column='Global X_Y', value=global_x_and_y)
new_nodes_ids_df.head()

NameError: name 'Basemap' is not defined

## Step 2: Local Projection
- The projection above considered the entire globe as the canvas, 
- we will convert it to a local canvas with the lower left point as origin

In [11]:
# For local x and y
min_lat = 33.8017
max_lat = 34.5720
min_long = -118.6983
max_long = -117.7269

min_y, min_x = mp(min_long, min_lat)
max_y, max_x = mp(max_long, max_lat)

print("X : min = {}, max ={}, \nY : min = {}, max ={}".format(min_x, max_x, min_y ,max_y))

# considering the lower left of our bounding box as (0,0): origin
local_x_and_y=[]

for i, (way_id, n_ids, new_n_ids, lat_long, global_xy) in new_nodes_ids_df.iterrows():
    
    insert=[]
    for j in global_xy:
        
        # Percentage method
        x = round(100*j[0]/max_x, 5)
        y = round(100*j[1]/max_y, 5)
        
        insert.append((x,y))
    local_x_and_y.append(insert)
    
new_nodes_ids_df.insert(loc=5, column='Local X_Y', value=local_x_and_y)
new_nodes_ids_df.head()

NameError: name 'mp' is not defined

## Step 3: Create nodes and edges files

- Using the dataframe above I will create la_nodes.csv and la_edges.csv files
- __la_nodes.csv__ columns: 
    - old node id, new node_id, lat, long, x, y
- __la_edges.csv__ columns: 
    - edge_id, node_1_new_id, node_1x, node_1y, node_2_new_id, node_2x, node_2y 


In [ ]:
# LA nodes:
old_id, new_id =[],[]
lat, long, x, y=[], [], [], []

for i, (w_id, n_ids, new_n_ids, lat_long,  global_xy, local_x_y) in new_nodes_ids_df.iterrows():
    for j in range(len(n_ids)):
        old_id.append(n_ids[j])
        new_id.append(new_n_ids[j])
        lat.append(lat_long[j][0])
        long.append(lat_long[j][1])
        x.append(local_x_y[j][0])
        y.append(local_x_y[j][1])
        
node_data = {'old_node_id': old_id, 'new_node_id': new_id, 'lat': lat , 'long': long, 'X':x, 'Y':y}

df_nodes = pd.DataFrame(data=node_data)
df_nodes.to_csv('./other_files/la_nodes.csv', index = None)

In [ ]:
node_file = pd.read_csv('./other_files/la_nodes.csv')
node_file.head()

In [ ]:
edge_id=[]
node_1_new_id, node_1x, node_1y =[],[],[]
node_2_new_id, node_2x, node_2y = [],[],[]

# To assign edge id
# Edge id starts from 0 and add 1 along the way
counter=0

for i, (w_id, n_ids, new_n_ids, lat_long,  global_xy, local_x_y) in new_nodes_ids_df.iterrows():
    
    # len(n_ids) is going to be at least 2
    for j in range(1,len(n_ids)):
        edge_id.append(counter)
        node_1_new_id.append(new_n_ids[j-1])
        node_1x.append(local_x_y[j-1][0])
        node_1y.append(local_x_y[j-1][1])
        
        node_2_new_id.append(new_n_ids[j])
        node_2x.append(local_x_y[j][0])
        node_2y.append(local_x_y[j][1])
        counter+=1
        
edge_data = {'edge_id': edge_id, 'node_1_id': node_1_new_id, 'node_1x': node_1x , 'node1y': node_1y,
             'node_2_id':node_2_new_id, 'node_2x':node_2x, 'node_2y':node_2y}
df_edges = pd.DataFrame(data=edge_data)

df_edges.to_csv('./other_files/la_edges.csv', index = None)

## Step 4: Plot graph using edges
- Why plot a graph? 
    - It gives fine control and power over what we can do with the data

In [ ]:
# we are actually plotting a graph here from nodes and edges

fig, axes = plt.subplots(figsize=(20,20), dpi=200)

# to not make it super tight
axes.set_xlim([-5, 105])
axes.set_ylim([-5, 105])

# Plotting just using the edges
for i, (edge_id,node_1_id,node_1x,node_1y,node_2_id,node_2x,node_2y) in tet.iterrows():
        
    x_c = [node_1x, node_2x]
    y_c = [node_1y, node_2y]
    
    axes.plot(x_c,y_c,'b-',linewidth=0.3)
           
fig.savefig('./images/plot_newtwork.png')

## Next Steps:
1. Overlay the road network with topology graph to make a better visualization (Basemap provides tools for this)
2. Overlay nodes of interest
3. Some applications may require construction of adjacency matrix (Since we already have nodes and edges files, creating adjacency matrix should not be too difficult)

#### Considerations while creating an adjacency matrix
- How to encode the information on one way and two way roads? : Directed vs. Undirected graph
- start with simple version, if an edge exists between the 2 nodes, the value is 1 else it is 0
- nodes x nodes dimension

#### What I did:
- I plotted sensor locations by calculating the distance to the nearest road segment as an overlay on top of the map above

## References: 
1. https://pubs.usgs.gov/pp/1395/report.pdf

__I'm sorry its not perfect__
- Dependency management sucks

#### An example of overlay
- This is a flat style, other overlays with topological maps are also available

In [ ]:
# Create a base layer map with the bounding box in 4 different styles
fig_1, axes_1 = plt.subplots(figsize=(10,10), dpi=100)
axes_1.set_xlim([-5, 105])
axes_1.set_ylim([-5, 105])
m = Basemap(llcrnrlon = -118.6983,
            llcrnrlat = 33.8017,
            urcrnrlon = -117.7269,
            urcrnrlat = 34.5720,
            projection='cea',
            resolution='h',
            fix_aspect = False,
            ax=axes_1)

m.drawmapboundary(fill_color='#A6CAE0', linewidth=0)
m.fillcontinents(color='brown', alpha=0.6, lake_color='grey')
m.drawcoastlines(linewidth=0.1, color="white")
fig_1.savefig('./images/ base_layer.png')